### 导入模拟数据

使用模拟数据，测试模型

In [1]:
import os
import numpy as np

os.chdir("/home/num_24/桌面/mywork/github/TFDeepSurv")
os.getcwd()

np.random.seed(10)

from dataset import SimulatedData
import LDeepSurv as LDS

data_config = SimulatedData(10000, num_features = 9)
data = data_config.generate_data(2000)
sort_idx = np.argsort(data['t'])[::-1]
data['x'] = data['x'] [sort_idx]
data['e'] = data['e'] [sort_idx]
data['t'] = data['t'] [sort_idx]

In [2]:
n_features = data['x'].shape[1]
print("features: ", n_features)
model = LDS.LDeepSurv(n_features, [7,5,3], 1, learning_rate=0.001, activation='tanh', L2_reg=0.01)

model.train(data['x'], {'e': data['e'], 't': data['t']}, num_epoch=5000)

features:  9
-------------------------------------------------
training steps 1: loss=9681.19.

CI on train set: 0.493198.

-------------------------------------------------
training steps 51: loss=9681.15.

CI on train set: 0.499988.

-------------------------------------------------
training steps 101: loss=9681.1.

CI on train set: 0.504897.

-------------------------------------------------
training steps 151: loss=9681.01.

CI on train set: 0.507577.

-------------------------------------------------
training steps 201: loss=9680.78.

CI on train set: 0.509831.

-------------------------------------------------
training steps 251: loss=9680.01.

CI on train set: 0.512087.

-------------------------------------------------
training steps 301: loss=9676.3.

CI on train set: 0.515305.

-------------------------------------------------
training steps 351: loss=9558.02.

CI on train set: 0.532579.

-------------------------------------------------
training steps 401: loss=9619.41.

CI 

-------------------------------------------------
training steps 3651: loss=8359.84.

CI on train set: 0.855737.

-------------------------------------------------
training steps 3701: loss=8359.2.

CI on train set: 0.855757.

-------------------------------------------------
training steps 3751: loss=8358.04.

CI on train set: 0.855775.

-------------------------------------------------
training steps 3801: loss=8363.82.

CI on train set: 0.854884.

-------------------------------------------------
training steps 3851: loss=8351.05.

CI on train set: 0.856768.

-------------------------------------------------
training steps 3901: loss=8371.73.

CI on train set: 0.854943.

-------------------------------------------------
training steps 3951: loss=8348.91.

CI on train set: 0.856247.

-------------------------------------------------
training steps 4001: loss=8340.02.

CI on train set: 0.856335.

-------------------------------------------------
training steps 4051: loss=8360.2.

CI o

In [3]:
N = 10
print(data['x'][0:N])
print(data['e'][0:N])
print(data['t'][0:N])

[[ 0.54264128 -0.95849609  0.26729646  0.49760777 -0.00298598 -0.55040669
  -0.60387427  0.52106142 -0.66177833]
 [-0.82332039  0.37071964  0.90678668 -0.99210346  0.02438453  0.62524194
   0.22505213  0.44351062 -0.41624787]
 [ 0.83554822  0.42915156  0.08508874 -0.71565992 -0.25331849  0.34826723
  -0.11633365 -0.13197201  0.23553395]
 [ 0.02627649  0.30079436  0.20207791  0.61044639  0.0432943   0.81729776
  -0.36152783 -0.81908131 -0.39859989]
 [-0.77203125  0.65736264 -0.90620738  0.25257429  0.09517231  0.638574
  -0.6021049   0.71370059 -0.29669473]
 [ 0.5092954  -0.40807658  0.76787293 -0.34897673 -0.66996819 -0.21494152
  -0.81307924  0.64221132 -0.69769597]
 [-0.2317711   0.88852143  0.97525096 -0.08739091  0.6522457  -0.49725172
   0.19474329  0.80566353  0.0691159 ]
 [ 0.18040273 -0.92143649 -0.28563648 -0.84077382 -0.38908017 -0.33856139
   0.54766059 -0.92008156 -0.14101565]
 [-0.37014624  0.2729823  -0.30730569 -0.91380531  0.75983036  0.52648115
   0.75619328 -0.1649817

In [5]:
sum(data['e']) 

1356

In [10]:
data['e'].shape[0]

2000

### 测试模型

In [5]:
n_features = data['x'].shape[1]
print("features: ", n_features)
model = LDeepSurv.LDeepSurv(n_features, [10, 5], 1, activation='tanh', L2_reg=0.001, L1_reg=0.001)

features:  9


-------------------------------------------------
training steps 1: loss=8943.11.

CI on train set: 0.500538.

-------------------------------------------------
training steps 21: loss=8943.07.

CI on train set: 0.501045.

-------------------------------------------------
training steps 41: loss=8943.07.

CI on train set: 0.501045.

-------------------------------------------------
training steps 61: loss=8943.07.

CI on train set: 0.501045.

-------------------------------------------------
training steps 81: loss=8943.07.

CI on train set: 0.501045.

-------------------------------------------------
training steps 101: loss=8943.07.

CI on train set: 0.501045.

-------------------------------------------------
training steps 121: loss=8943.07.

CI on train set: 0.501045.

-------------------------------------------------
training steps 141: loss=8943.07.

CI on train set: 0.501045.

-------------------------------------------------
training steps 161: loss=8943.07.

CI on train set: 